<a target="_blank" href="https://colab.research.google.com/github/pywavelet/pywavelet/blob/main/docs/runtime.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Runtime Comparisons

In [2]:
import numpy as np
from timeit import timeit
from pywavelet.types import FrequencySeries


def generate_freq_domain_signal(
    ND, f0=20.0, dt=0.0125, A=2
) -> FrequencySeries:
    """
    Generates a frequency domain signal.

    Parameters:
    ND (int): Number of data points.
    f0 (float): Frequency of the signal. Default is 20.0.
    dt (float): Time step. Default is 0.0125.
    A (float): Amplitude of the signal. Default is 2.

    Returns:
    FrequencySeries: The generated frequency domain signal.
    """
    ts = np.arange(0, ND) * dt
    y = A * np.sin(2 * np.pi * f0 * ts)
    yf = FrequencySeries(y, ts)
    return yf

In [15]:
from pywavelet.transforms.numpy.forward.from_freq import (
    transform_wavelet_freq_helper,
)
from pywavelet.transforms.phi_computer import phitilde_vec_norm
from timeit import timeit

NF_values = [8, 16, 32, 64, 128]
results = {}

for Nf in NF_values:

    Nt = Nf * 2
    ND = Nf * Nt
    print(f"ND: {ND}, Nf: {Nf}, Nt: {Nt}")
    yf = generate_freq_domain_signal(ND)
    phif = phitilde_vec_norm(Nf, Nt, d=4.0)

    # Warm up run
    transform_wavelet_freq_helper(yf.data, Nf, Nt, phif)

    # Timeit run
    time_taken = timeit(
        lambda: transform_wavelet_freq_helper(yf.data, Nf, Nt, phif), number=5
    )
    results[ND] = time_taken

ND: 128, Nf: 8, Nt: 16
ND: 512, Nf: 16, Nt: 32
ND: 2048, Nf: 32, Nt: 64
ND: 8192, Nf: 64, Nt: 128
ND: 32768, Nf: 128, Nt: 256


In [16]:
results

{128: 9.749794844537973e-05,
 512: 0.00019677798263728619,
 2048: 0.0004649469628930092,
 8192: 0.0012474380200728774,
 32768: 0.003837117925286293}